###Carregamento do modelo

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-14B-Instruct-AWQ"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/homeLocal/sofia/venv_awq/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/homeLocal/sofia/venv_awq/lib/python3.12/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
We suggest you to set `torch_dtype=torch.float16` for better efficiency on CUDA/XPU with AWQ.
/homeLocal/sofia/venv_awq/lib/python3.12/site-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue

###Prática 1

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code1-1.py", "Solucao/solucao-code1.py", "lint/code1-1-lint.pre", "ast/code1-1-ast.pre", "Esqueleto/esqueleto-code1.py", "Saida/saida1.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code1.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code1.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p02.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code1.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code1-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code1-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code1-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida1.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para calcular o período em que o estudante se encontra no curso, considere que cada ano tem 2 períodos letivos;
    - O programa deve informar a idade provável pois não tem informação sobre o mês de nascimento.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
    do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código inicial fornecido pelo professor. Vamos analisar se o aluno preencheu os campos solicitados no código base:

1. **Nome do aluno**: O aluno preencheu o campo com "Ana Maria".
2. **Matrícula**: O aluno preencheu o campo com "245637".
3. **Data**: O aluno preencheu o campo com "10/09/2025".
4. **Breve descrição do programa**: O aluno substituiu o comentário do código base por "substitua essa linha por um comentário dizendo o que o programa faz". Isso indica que o aluno não adicionou uma descrição específica do que o programa faz.

### Boas Práticas

Aqui estão as observações do pylint que devem ser consideradas:

1. **Nomes de variáveis**: O nome da constante `anoAtual` não segue o estilo UPPER_CASE recomendado para constantes em Python. É recomendado renomear para `ANO_ATUAL`.
2. **Ponto e vírgula no final das instruções**: O uso de ponto e vírgula no final das instruções é desnecessário em Python e pode ser removido.
3. 

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code1-2.py", "Solucao/solucao-code1.py", "lint/code1-2-lint.pre", "ast/code1-2-ast.pre", "Esqueleto/esqueleto-code1.py", "Saida/saida1.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code1.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code1.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p02.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code1.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code1-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code1-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code1-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida1.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para calcular o período em que o estudante se encontra no curso, considere que cada ano tem 2 períodos letivos;
    - O programa deve informar a idade provável pois não tem informação sobre o mês de nascimento.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno Bruna Souza foi baseado no código inicial fornecido pelo professor. Vamos analisar se o aluno preencheu os campos solicitados no cabeçalho e se adicionou uma breve descrição do programa.

**Cabeçalho:**
- **Nome do aluno:** Bruna Souza (preenchido corretamente)
- **Matrícula:** 144657 (preenchido corretamente)
- **Data:** 27/08/2025 (preenchido corretamente)
- **Breve descrição:** "O programa identifica a idade e em qual período o aluno entrou na UFV." (preenchido corretamente)

**Conclusão:** O aluno preencheu todos os campos solicitados no cabeçalho e adicionou uma breve descrição do programa.

### Boas Práticas

Aqui estão as observações do pylint que devem ser consideradas para melhorar o código:

1. **Nomes de variáveis:** O nome da constante `anoAtual` não segue o estilo UPPER_CASE recomendado para constantes. É recomendado renomear para `ANO_ATUAL`.

2. **Linhas muito longas:** A linha de impressão está muito longa e pode ser quebrada

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code1-3.py", "Solucao/solucao-code1.py", "lint/code1-3-lint.pre", "ast/code1-3-ast.pre", "Esqueleto/esqueleto-code1.py", "Saida/saida1.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code1.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code1.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p02.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code1.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code1-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code1-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code1-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida1.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para calcular o período em que o estudante se encontra no curso, considere que cada ano tem 2 períodos letivos;
    - O programa deve informar a idade provável pois não tem informação sobre o mês de nascimento.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno segue o padrão básico solicitado pelo professor, incluindo a declaração de variáveis e a leitura de entradas do usuário. No entanto, há algumas diferenças notáveis entre o código base fornecido pelo professor e o código implementado pelo aluno:

1. **Preenchimento do cabeçalho**:
   - **Nome do aluno**: O aluno preencheu corretamente com "Bárbara Silveira".
   - **Matrícula**: O aluno preencheu corretamente com "144659".
   - **Data**: O aluno preencheu corretamente com "27/08/2025".
   - **Breve descrição**: O aluno descreveu o programa como "O programa fala idade, período e qual período de ingresso do aluno na UFV."

2. **Comentários**: O aluno adicionou comentários para cada linha de entrada, o que é uma boa prática, mas poderia ser mais detalhado para melhorar a compreensão do código.

### Boas Práticas

Aqui estão as observações gerais sobre o código do aluno, baseadas nas recomendações do pylint:

1. **Nomes de Variáveis**: O nome da c

###Prática 2

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code2-1.py", "Solucao/solucao-code2.py", "lint/code2-1-lint.pre", "ast/code2-1-ast.pre", "Esqueleto/esqueleto-code2.py", "Saida/saida2.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code2.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code2.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p03.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code2.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code2-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code2-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code2-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida2.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para qualquer valor diferente de 1, a escolha é a opção 2.
    - A tarefa do aluno é incluir um único if-else no código.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Feedback para o Aluno

#### Descrição
O código base fornecido pelo professor contém comentários no topo para que o aluno preencha o nome, matrícula e data, além de adicionar uma breve descrição do que o programa faz. No código do aluno, observei que:

- **Nome, Matrícula e Data:** O aluno preencheu corretamente o nome ("Armando Batista"), a matrícula ("528694") e a data ("03/09/2025").
- **Breve Descrição:** O aluno adicionou uma breve descrição do que o programa faz, que é "Este programa calcula a área e o perímetro de figuras regulares".

#### Boas Práticas
As observações do pylint apontadas para o código do aluno são:

- **Unnecessary parens after 'if' keyword:** Evite colocar parênteses desnecessários após a palavra-chave `if`. Por exemplo, `if (opcao == 1):` deve ser escrito como `if opcao == 1:`.

#### Lógica e Concordância com o Roteiro
A implementação do aluno está correta e segue a lógica definida no roteiro. O aluno utilizou corretamente um comando `if-else` para separar 

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code2-2.py", "Solucao/solucao-code2.py", "lint/code2-2-lint.pre", "ast/code2-2-ast.pre", "Esqueleto/esqueleto-code2.py", "Saida/saida2.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code2.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code2.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p03.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code2.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code2-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code2-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code2-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida2.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para qualquer valor diferente de 1, a escolha é a opção 2.
    - A tarefa do aluno é incluir um único if-else no código.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Feedback para o Aluno

#### Descrição
O código base fornecido pelo professor contém comentários no topo para que o aluno preencha o nome, matrícula e data, além de adicionar uma breve descrição do que o programa faz. No código do aluno, observei que:

- **Nome, Matrícula e Data:** Estes campos foram preenchidos corretamente.
- **Breve Descrição:** O aluno adicionou uma breve descrição do que o programa faz, indicando que o programa calcula a área e o perímetro de figuras regulares (círculo ou retângulo).

#### Boas Práticas
O pylint apontou algumas observações que podem ser melhorias no código do aluno:

- **Nomes de Variáveis:** O nome da variável `figura` foi usado duas vezes, uma vez para o círculo e outra para o retângulo. É recomendável usar nomes de variáveis distintos para evitar confusão, especialmente se o código for expandido ou modificado no futuro.
- **Estilo de Nomenclatura:** Embora o pylint não tenha apontado problemas específicos com o estilo de nomenclatura, é uma 

###Prática 3

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code3-1.py", "Solucao/solucao-code3.py", "lint/code3-1-lint.pre", "ast/code3-1-ast.pre", "Esqueleto/esqueleto-code3.py", "Saida/saida3.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code3.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code3.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p04.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code3.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code3-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code3-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code3-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida3.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para crianças com menos de 4 anos e idosos com 65 anos ou mais, o valor do ingresso é 0 reais e é sem acesso.
    - Para crianças de 4 a 12 anos, o valor do ingresso deve ser 75 reais, de 13 a 17 deve ser 85 reais e de 18 ou mais, 115 reais.
    - A altura só deve ser solicitada quando é necessária para definir o tipo de ingresso, ou seja, apenas nos casos do tópico anterior, sendo acesso restrito para menos de 1.50 de altura (aplicando desconto de 10% no valor do ingresso) e acesso total para altura maior ou igual a 1.50.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código inicial fornecido pelo professor. Vamos analisar se o aluno preencheu os campos obrigatórios e se adicionou uma descrição adequada.

**Campos Preenchidos:**
- **Nome do Aluno:** Bianca Andrade
- **Matrícula:** 101457
- **Data:** 10/09/2025

**Descrição do Programa:**
O aluno adicionou uma breve descrição: "O programa gerencia o acesso ao parque de acordo com idade e altura."

### Boas Práticas

Aqui estão as observações do pylint, adaptadas para uma linguagem mais clara e direta:

1. **Indentação Consistente:** O código possui problemas de indentação inconsistentes. É importante manter uma indentação consistente, geralmente 4 espaços por nível de indentação.
2. **Nomes de Variáveis:** Os nomes das variáveis `ingresso` e `tipo_acesso` não seguem as convenções de nomes recomendadas. É melhor usar nomes que indiquem claramente o propósito da variável, como `valor_ingresso` e `tipo_acesso`.
3. **Uso de Variáveis Antes da At

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code3-2.py", "Solucao/solucao-code3.py", "lint/code3-2-lint.pre", "ast/code3-2-ast.pre", "Esqueleto/esqueleto-code3.py", "Saida/saida3.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code3.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code3.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p04.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code3.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code3-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code3-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code3-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida3.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para crianças com menos de 4 anos e idosos com 65 anos ou mais, o valor do ingresso é 0 reais e é sem acesso.
    - Para crianças de 4 a 12 anos, o valor do ingresso deve ser 75 reais, de 13 a 17 deve ser 85 reais e de 18 ou mais, 115 reais.
    - A altura só deve ser solicitada quando é necessária para definir o tipo de ingresso, ou seja, apenas nos casos do tópico anterior, sendo acesso restrito para menos de 1.50 de altura (aplicando desconto de 10% no valor do ingresso) e acesso total para altura maior ou igual a 1.50.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno segue o esqueleto básico fornecido pelo professor, mas há algumas diferenças notáveis:

1. **Preenchimento do cabeçalho:**
   - **Nome do aluno:** O aluno preencheu corretamente com "Caio Pires".
   - **Matrícula:** O aluno preencheu corretamente com "115747".
   - **Data:** O aluno preencheu corretamente com "10/09/2025".
   - **Breve descrição:** O aluno adicionou uma descrição adequada: "Dependendo da idade e altura é definido o acesso aos brinquedos".

2. **Comentários:** O aluno não incluiu comentários adicionais no código, o que é aceitável, mas poderia ajudar na compreensão do código para outros desenvolvedores ou para si mesmo em futuras revisões.

### Boas Práticas

1. **Nomes de Variáveis:**
   - Os nomes das variáveis `tipo` e `valor` devem ser em letras minúsculas, conforme a convenção de nomes do Python. Por exemplo, `tipo_ingresso` e `valor_ingresso`.

2. **Simplificação de Condições:**
   - As condições de idade podem ser simp

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code3-3.py", "Solucao/solucao-code3.py", "lint/code3-3-lint.pre", "ast/code3-3-ast.pre", "Esqueleto/esqueleto-code3.py", "Saida/saida3.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code3.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code3.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p04.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code3.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code3-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code3-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code3-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida3.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Para crianças com menos de 4 anos e idosos com 65 anos ou mais, o valor do ingresso é 0 reais e é sem acesso.
    - Para crianças de 4 a 12 anos, o valor do ingresso deve ser 75 reais, de 13 a 17 deve ser 85 reais e de 18 ou mais, 115 reais.
    - A altura só deve ser solicitada quando é necessária para definir o tipo de ingresso, ou seja, apenas nos casos do tópico anterior, sendo acesso restrito para menos de 1.50 de altura (aplicando desconto de 10% no valor do ingresso) e acesso total para altura maior ou igual a 1.50.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código inicial fornecido pelo professor. Vamos analisar se o aluno preencheu os campos obrigatórios e se adicionou uma descrição adequada.

**Campos Preenchidos:**
- **Nome do Aluno:** Miguel Pires
- **Matrícula:** 115749
- **Data:** 10/09/2025

**Descrição do Programa:**
O aluno adicionou uma breve descrição: "A idade e altura da pessoa define o valor e tipo de ingresso".

### Boas Práticas

Aqui estão as observações do pylint e as boas práticas relacionadas:

1. **Nomes de Variáveis:**
   - Os nomes das variáveis `valor` devem seguir o estilo de nomes de constantes em Python, que é em letras maiúsculas. Portanto, `VALOR` seria a maneira correta de nomear essa variável.

2. **Espaços em Branco:**
   - O pylint aponta um erro de espaços em branco no final da linha 9. É importante remover esses espaços para manter um código limpo e livre de erros.

3. **Docstrings:**
   - O pylint indica que falta uma docstring no módulo. Uma d

###Prática 4

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code4-1.py", "Solucao/solucao-code4.py", "lint/code4-1-lint.pre", "ast/code4-1-ast.pre", "Esqueleto/esqueleto-code4.py", "Saida/saida4.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code4.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code4.py']}

Para resolver a atividade, o professor fornece o seguinte código base para o aluno completar:

{codes['Esqueleto/esqueleto-code4.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code4-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code4-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code4-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida4.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se a idade do aluno for >= a 16 anos, ele pode participar e não precisa solicitar tempo de escolinha.
    - Em caso negativo, se o tempo de escolhinha for >= 5 anos, ele pode participar.
    - Em caso negativo, se a soma do tempo de escolinha e idade for >= 18 anos, ele pode participar.
    - Caso nenhuma das condições seja atendida, o aluno não pode participar.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código base fornecido pelo professor, mas há algumas diferenças notáveis:

1. **Preenchimento do cabeçalho:**
   - **Nome:** O aluno preencheu o campo "Nome" com "Cássio Pereira".
   - **Matrícula:** O aluno preencheu o campo "Matrícula" com "125478".
   - **Data:** O aluno preencheu o campo "Data" com "17/09/25".
   - **Descrição:** O aluno adicionou uma breve descrição do que o programa faz: "O programa define condições para solucionar um problema."

2. **Comentários:**
   - O aluno incluiu comentários no código, o que é uma boa prática, mas eles não são necessariamente coerentes com o roteiro fornecido pelo professor.

### Boas Práticas

1. **Pylint Observações:**
   - **Trailing Whitespace:** O código contém espaços em branco no final de algumas linhas, o que pode ser removido para manter a consistência.
   - **Missing Module Docstring:** O código não possui uma descrição do módulo, o que é uma boa prática para documentar 

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code4-2.py", "Solucao/solucao-code4.py", "lint/code4-2-lint.pre", "ast/code4-2-ast.pre", "Esqueleto/esqueleto-code4.py", "Saida/saida4.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code4.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code4.py']}

Para resolver a atividade, o professor fornece o seguinte código base para o aluno completar:

{codes['Esqueleto/esqueleto-code4.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code4-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code4-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code4-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida4.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se a idade do aluno for >= a 16 anos, ele pode participar e não precisa solicitar tempo de escolinha.
    - Em caso negativo, se o tempo de escolhinha for >= 5 anos, ele pode participar.
    - Em caso negativo, se a soma do tempo de escolinha e idade for >= 18 anos, ele pode participar.
    - Caso nenhuma das condições seja atendida, o aluno não pode participar.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Feedback para a Atividade de Programação INF100

#### Descrição
O código base fornecido pelo professor inclui campos para o nome do aluno, matrícula, data e uma breve descrição do programa. Vamos analisar se o aluno preencheu esses campos corretamente.

**Nome, Matrícula e Data:**
- **Aluno:** Gabriel Fernandes
- **Matrícula:** 123456
- **Data:** 18/08/2025

Esses campos foram preenchidos corretamente.

**Breve Descrição:**
- **Descrição Original:** (breve comentário dizendo o que o programa faz)
- **Descrição do Aluno:** O código recebe a idade e verifica se os alunos podem ou não participar do campeonato

A descrição do aluno foi adicionada e é clara sobre o objetivo do programa.

#### Boas Práticas
Aqui estão as observações do pylint, adaptadas para uma linguagem mais simples e direta:

1. **Indentação:** O código possui problemas de indentação. Algumas linhas têm 3 ou 7 espaços, quando deveriam ter 4 ou 8 espaços, respectivamente. É importante manter a consistência na indentaçã

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code4-3.py", "Solucao/solucao-code4.py", "lint/code4-3-lint.pre", "ast/code4-3-ast.pre", "Esqueleto/esqueleto-code4.py", "Saida/saida4.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code4.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code4.py']}

Para resolver a atividade, o professor fornece o seguinte código base para o aluno completar:

{codes['Esqueleto/esqueleto-code4.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code4-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code4-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code4-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida4.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se a idade do aluno for >= a 16 anos, ele pode participar e não precisa solicitar tempo de escolinha.
    - Em caso negativo, se o tempo de escolhinha for >= 5 anos, ele pode participar.
    - Em caso negativo, se a soma do tempo de escolinha e idade for >= 18 anos, ele pode participar.
    - Caso nenhuma das condições seja atendida, o aluno não pode participar.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código base fornecido pelo professor, mas há algumas diferenças notáveis:

1. **Preenchimento do cabeçalho:**
   - **Nome do aluno:** O aluno preencheu o campo "Nome do aluno" com "Mara Silva".
   - **Matrícula:** O aluno preencheu o campo "Matrícula" com "125478".
   - **Data:** O aluno preencheu o campo "Data" com "10/09/2025".
   - **Breve descrição:** O aluno não adicionou uma breve descrição do que o programa faz. O comentário do código base permanece inalterado.

### Boas Práticas

Aqui estão as observações do pylint, adaptadas para uma linguagem mais amigável e relevante para o contexto:

1. **Uso de f-strings:** O pylint sugeriu que o aluno considere usar f-strings para formatar as strings. F-strings são mais legíveis e eficientes do que o método de formatação tradicional usando `%`. Por exemplo, em vez de `'\n%s, você pode jogar o campeonato pela idade: %d anos' % (nome, idade)`, o aluno poderia usar `f'\n{nome}, você

###Prática 5

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code5-1.py", "Solucao/solucao-code5.py", "lint/code5-1-lint.pre", "ast/code5-1-ast.pre", "Esqueleto/esqueleto-code5.py",  "Esqueleto/esqueleto-code5-2.py", "Saida/saida5.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code5.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code5.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p06.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code5.py']}

E fornece também o código repetitivo.py, que mostra como usar o comando while:

{codes['Esqueleto/esqueleto-code5-2.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code5-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code5-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code5-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida5.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O programa deve executar pelo menos uma vez, perguntando, a cada vez, se o usuário deseja repetir a execução.
    - A lógica do repetitivo.py pode ser aplicada pelo aluno, assim como ele pode usar outra, desde que cumpra a tarefa
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Feedback para o Aluno

#### Descrição
O código base fornecido pelo professor inclui campos para o nome do aluno, matrícula, data e uma breve descrição do programa. No código do aluno, observe se esses campos foram preenchidos corretamente:

- **Nome do aluno:** Fabricio Kenji
- **Matrícula:** 142587
- **Data:** 24-09-2025

Além disso, o aluno deve ter adicionado uma breve descrição do que o programa faz. No código base, o comentário é:
```python
# Substitua estas 2 linhas por um breve comentário dizendo o que o 
# programa faz, após você ter concluído a terceira tarefa do roteiro
```

No código do aluno, o comentário é:
```python
# Este programa calcula as raízes de uma equação quantas vezes o usuário desejar
```

#### Boas Práticas
O pylint apontou algumas observações que podem ser melhorias no código do aluno:

1. **Trailing newlines:** Evite deixar linhas em branco no final do arquivo.
2. **Missing module docstring:** Adicione uma descrição do módulo no início do código.
3. **In

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code5-2.py", "Solucao/solucao-code5.py", "lint/code5-2-lint.pre", "ast/code5-2-ast.pre", "Esqueleto/esqueleto-code5.py",  "Esqueleto/esqueleto-code5-2.py", "Saida/saida5.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code5.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code5.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p06.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code5.py']}

E fornece também o código repetitivo.py, que mostra como usar o comando while:

{codes['Esqueleto/esqueleto-code5-2.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code5-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code5-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code5-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida5.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O programa deve executar pelo menos uma vez, perguntando, a cada vez, se o usuário deseja repetir a execução.
    - A lógica do repetitivo.py pode ser aplicada pelo aluno, assim como ele pode usar outra, desde que cumpra a tarefa
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno segue o formato básico fornecido pelo professor, incluindo os campos para o nome do aluno, matrícula e data. No entanto, o aluno não preencheu esses campos, mantendo-os como no código base. Além disso, o aluno não adicionou uma breve descrição do que o programa faz, deixando o comentário do código base inalterado.

### Boas Práticas

1. **Nome do Módulo**: O nome do módulo (`code5-2.py`) não segue a convenção de nomes em snake_case. É recomendado que o nome do arquivo seja alterado para `p06.py`, conforme solicitado no roteiro.

2. **Nomes de Variáveis**: O nome da variável `cont` não segue a convenção de nomes em UPPER_CASE para constantes. Embora `cont` não seja uma constante, é uma boa prática usar nomes descritivos para variáveis. Por exemplo, `continuar` ou `repetir` poderiam ser melhores nomes.

3. **Trailing Whitespace**: O código contém espaços em branco no final de algumas linhas. É importante remover esses espaços para manter um có

###Prática 6

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code6-1.py", "Solucao/solucao-code6.py", "lint/code6-1-lint.pre", "ast/code6-1-ast.pre", "Esqueleto/esqueleto-code6.py", "Saida/saida6.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code6.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code6.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p07.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code6.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code6-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code6-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code6-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida6.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se o palpite do usuário estiver dentro do intervalo, deve ser considerado como palpite válido e incrementado no contador.
    - Adicionalmente, se o palpite estiver errado, o código deve retornar se o número sorteado é maior ou menor do que o palpite e definir novo intervalo com base no palpite.
    - A semente só deve ser inicializada uma vez e o intervalo deve voltar a ser [1,100] caso o usuário acerte o número sorteado e decida jogar novamente.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código base fornecido pelo professor, mas há algumas diferenças notáveis:

1. **Preenchimento de Informações:**
   - **Nome, Matrícula e Data:** O aluno preencheu o campo de nome (`Lucas Marques`), matrícula (`121475`) e data (`29/09/2025`).
   - **Breve Descrição:** O aluno adicionou uma breve descrição do que o programa faz, embora não seja muito detalhada. A descrição é: `Escreva aqui um breve comentário sobre o programa solução d`.

2. **Conformidade com o Código Base:**
   - O aluno seguiu a estrutura básica fornecida pelo professor, incluindo a importação da biblioteca `random`, a definição do intervalo, a mensagem inicial, a inicialização da semente e o sorteio do número aleatório.

### Boas Práticas

Aqui estão as observações apontadas pelo pylint, reformuladas de forma clara e didática:

1. **Indentação Consistente:**
   - O código possui problemas de indentação inconsistentes. Por exemplo, algumas linhas têm 9 espaço

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code6-2.py", "Solucao/solucao-code6.py", "lint/code6-2-lint.pre", "ast/code6-2-ast.pre", "Esqueleto/esqueleto-code6.py", "Saida/saida6.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code6.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code6.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p07.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code6.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code6-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code6-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code6-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida6.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se o palpite do usuário estiver dentro do intervalo, deve ser considerado como palpite válido e incrementado no contador.
    - Adicionalmente, se o palpite estiver errado, o código deve retornar se o número sorteado é maior ou menor do que o palpite e definir novo intervalo com base no palpite.
    - A semente só deve ser inicializada uma vez e o intervalo deve voltar a ser [1,100] caso o usuário acerte o número sorteado e decida jogar novamente.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código base fornecido pelo professor, mas há algumas diferenças notáveis:

1. **Informações de cabeçalho**: O aluno preencheu o campo de nome, matrícula e data, mas não adicionou uma breve descrição do que o programa faz. O cabeçalho do aluno está assim:
    ```python
    # Nome: Maria Souza
    # Matrícula: 125784
    # Data:	31/09/25	   
    # Um número é sorteado e o operador deve tentar adivinhar como uma
    # brincadeira de criança 
    ```

2. **Descrição do programa**: O aluno incluiu uma breve descrição do que o programa faz, mas ela poderia ser mais detalhada para refletir completamente o funcionamento do jogo.

### Boas Práticas

1. **Nomes de variáveis**: O pylint apontou que os nomes das constantes `linf`, `lsup` e `palpites` não seguem a convenção de UPPER_CASE. É recomendável que esses nomes sejam alterados para `LINF`, `LSUP` e `PALPITES` respectivamente.

2. **Remoção de `elif` após `break`**: O pylint apontou

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code6-3.py", "Solucao/solucao-code6.py", "lint/code6-3-lint.pre", "ast/code6-3-ast.pre", "Esqueleto/esqueleto-code6.py", "Saida/saida6.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code6.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code6.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p07.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code6.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code6-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code6-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code6-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida6.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Se o palpite do usuário estiver dentro do intervalo, deve ser considerado como palpite válido e incrementado no contador.
    - Adicionalmente, se o palpite estiver errado, o código deve retornar se o número sorteado é maior ou menor do que o palpite e definir novo intervalo com base no palpite.
    - A semente só deve ser inicializada uma vez e o intervalo deve voltar a ser [1,100] caso o usuário acerte o número sorteado e decida jogar novamente.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

/homeLocal/sofia/venv_awq/lib/python3.12/site-packages/PyPDF2/__init__.py:21: DeprecationWarning: PyPDF2 is deprecated. Please move to the pypdf library instead.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código base fornecido pelo professor, mas há algumas diferenças notáveis:

1. **Informações de cabeçalho**: O aluno preencheu o campo de nome, matrícula e data, mas não adicionou uma breve descrição do que o programa faz. O cabeçalho do aluno está assim:
    ```python
    # Nome: Maria Roberta
    # Matrícula: 125786
    # Data:	31/09/25	   
    # O usuário deve adivinhar o número sorteado
    ```

2. **Descrição do programa**: O aluno incluiu uma breve descrição do que o programa faz, mas ela poderia ser mais detalhada. A descrição atual é:
    ```python
    # O usuário deve adivinhar o número sorteado
    ```

### Boas Práticas

1. **Nomes de variáveis**: Os nomes das variáveis `linf`, `lsup` e `palpites` não estão em conformidade com as convenções de estilo Python (snake_case). É recomendado renomeá-las para `limite_inferior`, `limite_superior` e `contador_palpites`, respectivamente.

2. **Uso desnecessário de `elif` após `

###Prática 7

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code7-1.py", "Solucao/solucao-code7.py", "lint/code7-1-lint.pre", "ast/code7-1-ast.pre", "Esqueleto/esqueleto-code7.py", "Saida/saida7.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code7.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code7.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p08.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code7.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code7-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code7-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code7-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida7.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Como primeira tarefa, o programa deve ler uma semente inteira e, se ela for negativa, deve pedir o valor da semente novamente até que seja digitado um valor >=0.
    - Como segunda tarefa, ele deve percorrer o arranjo e contar quantos valores são maiores do que a média inteira já implementada pelo professor na solução
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

**Nome, Matrícula e Data:**
- **Nome:** O aluno preencheu o campo "Nome do aluno" com "Maria Souza".
- **Matrícula:** O aluno preencheu o campo "Matrícula" com "125784".
- **Data:** O aluno preencheu o campo "Data" com "08/10/25".

**Breve Descrição:**
- O aluno não substituiu o comentário do código base para adicionar uma breve descrição do que o programa faz. O comentário permanece inalterado.

### Boas Práticas

**Observações do Pylint:**

1. **Considerar o uso de `enumerate` em vez de iterar com `range` e `len`:**
   - **Linha 29:** No loop `for` que imprime os elementos do arranjo, o aluno poderia usar `enumerate` para simplificar o código.
   - **Linha 34:** No loop `for` que calcula a soma dos elementos do arranjo, o aluno poderia usar `enumerate` para simplificar o código.

2. **Nomes de constantes devem seguir o estilo UPPER_CASE:**
   - **Linhas 12 e 13:** Os nomes das constantes `linf` e `lsup` devem ser alterados para `LINF` e `LSUP`, respectivamente.
   - **

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code7-2.py", "Solucao/solucao-code7.py", "lint/code7-2-lint.pre", "ast/code7-2-ast.pre", "Esqueleto/esqueleto-code7.py", "Saida/saida7.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code7.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code7.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p08.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code7.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code7-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code7-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code7-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida7.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Como primeira tarefa, o programa deve ler uma semente inteira e, se ela for negativa, deve pedir o valor da semente novamente até que seja digitado um valor >=0.
    - Como segunda tarefa, ele deve percorrer o arranjo e contar quantos valores são maiores do que a média inteira já implementada pelo professor na solução
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

**Nome, Matrícula e Data:**
- **Nome:** O aluno preencheu o campo "Nome do aluno" com "Mirian Silva".
- **Matrícula:** O aluno preencheu o campo "Matrícula" com "99578".
- **Data:** O aluno preencheu o campo "Data" com "08/10/2025".

**Breve Descrição:**
- O aluno adicionou uma breve descrição no comentário do código, que diz: "O código verifica se o valor digitado é maior que zero e vê a média dos 25 números do arranjo... e depois imprime os números maiores que a média."

### Boas Práticas

**Observações do Pylint:**

1. **Considerar o uso de `enumerate` em vez de iterar com `range` e `len`:**
   - **Linha 34:** No loop `for i in range(0, len(A)):` considere usar `enumerate(A)` para tornar o código mais legível e evitar erros com `len(A)`.
   - **Linha 40:** No loop `for i in range(0, len(A)):` considere usar `enumerate(A)` para tornar o código mais legível e evitar erros com `len(A)`.

2. **Nomes de constantes devem seguir o estilo UPPER_CASE:**
   - **Linha 12:** O no

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code7-3.py", "Solucao/solucao-code7.py", "lint/code7-3-lint.pre", "ast/code7-3-ast.pre", "Esqueleto/esqueleto-code7.py", "Saida/saida7.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code7.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code7.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p08.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code7.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code7-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code7-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code7-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida7.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - Como primeira tarefa, o programa deve ler uma semente inteira e, se ela for negativa, deve pedir o valor da semente novamente até que seja digitado um valor >=0.
    - Como segunda tarefa, ele deve percorrer o arranjo e contar quantos valores são maiores do que a média inteira já implementada pelo professor na solução
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno Carla Márcia foi baseado no código fornecido pelo professor, com algumas modificações e adições. Vamos analisar se o aluno preencheu os campos obrigatórios e se adicionou uma descrição adequada.

**Campos Preenchidos:**
- **Nome do Aluno:** Carla Márcia
- **Matrícula:** 526487
- **Data:** 10/11/2025

**Descrição do Programa:**
O aluno adicionou uma breve descrição no cabeçalho do código:
```
# O programa confere quantos números são maiores do que a média dos valores de um array
```

Essa descrição é clara e concisa, indicando o propósito do programa.

### Boas Práticas

Aqui estão as observações do pylint, adaptadas para uma linguagem mais amigável e direcionada para iniciantes em programação:

1. **Uso de `enumerate`:**
   - **Observação:** Considere usar `enumerate` em vez de iterar com `range` e `len`.
   - **Explicação:** O uso de `enumerate` torna o código mais legível e evita erros comuns associados ao índice.

2. **Nomenclatura de Var

###Prática 8

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code8-1.py", "Solucao/solucao-code8.py", "lint/code8-1-lint.pre", "ast/code8-1-ast.pre", "Esqueleto/esqueleto-code8.py", "Saida/saida8.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code8.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code8.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p09.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code8.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code8-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code8-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code8-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida8.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve ordenar os elentos do arranjo A em ordem crescente sem poder usar a função sort, imprimindo esse arranjo ordenado depois.
    - Determinar e informar o valor da mediana: que é o elemento da posição do meio em caso de arranjo com tamanho ímpar ou a média aritmética dos dois valores centrais em caso de arranjo com tamanho par.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código base fornecido pelo professor. Vamos analisar se o aluno preencheu os campos obrigatórios e se adicionou uma breve descrição do programa.

**Campos Preenchidos:**
- **Nome do Aluno:** Maria Fernanda
- **Matrícula:** 145896
- **Data:** 22/10/2025

**Descrição do Programa:**
O aluno adicionou a seguinte descrição:
```
O programa faz um arranjo de 1 a 100 ficar ordenado
e exibe o valor da mediana deles.
```

Essa descrição é bastante simples e direta, mas poderia ser mais detalhada para refletir melhor o que o programa faz, incluindo a geração de um arranjo aleatório e a validação das entradas.

### Boas Práticas

O pylint apontou um erro de sintaxe relacionado à falta de indentação após a palavra-chave `else`. Vamos discutir isso e outras possíveis melhorias:

1. **Indentação após `else`:**
   - O código do aluno tem um erro de sintaxe na linha 51 onde a palavra-chave `else` não está seguida por um bloco indentado. Isso d

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code8-2.py", "Solucao/solucao-code8.py", "lint/code8-2-lint.pre", "ast/code8-2-ast.pre", "Esqueleto/esqueleto-code8.py", "Saida/saida8.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code8.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code8.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p09.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code8.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code8-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code8-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code8-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida8.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve ordenar os elentos do arranjo A em ordem crescente sem poder usar a função sort, imprimindo esse arranjo ordenado depois.
    - Determinar e informar o valor da mediana: que é o elemento da posição do meio em caso de arranjo com tamanho ímpar ou a média aritmética dos dois valores centrais em caso de arranjo com tamanho par.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código base fornecido pelo professor, com algumas modificações e adições. Vamos analisar se o aluno preencheu os campos obrigatórios e se adicionou uma descrição adequada.

**Campos Preenchidos:**
- **Nome do Aluno:** Matheus Jorge
- **Matrícula:** 236587
- **Data:** 22/10/2025

**Descrição do Programa:**
O aluno adicionou uma breve descrição no cabeçalho do código:
```
# O programa ordena um arranjo A e calcula a mediana dos elementos
```

Essa descrição é bastante adequada e concisa, indicando claramente o propósito do programa.

### Boas Práticas

Aqui estão as observações do pylint, reformuladas de forma mais clara e didática:

1. **Nomes de Variáveis e Constantes:**
   - Os nomes das constantes `linf` e `lsup` devem seguir o estilo UPPER_CASE, pois são constantes. Por exemplo, `LINF` e `LSUP`.
   - O nome do módulo `code8-2.py` não segue o estilo snake_case recomendado para nomes de arquivos Python. Ele deve ser renomeado

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code8-3.py", "Solucao/solucao-code8.py", "lint/code8-3-lint.pre", "ast/code8-3-ast.pre", "Esqueleto/esqueleto-code8.py", "Saida/saida8.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code8.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code8.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p09.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code8.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code8-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code8-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code8-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida8.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve ordenar os elentos do arranjo A em ordem crescente sem poder usar a função sort, imprimindo esse arranjo ordenado depois.
    - Determinar e informar o valor da mediana: que é o elemento da posição do meio em caso de arranjo com tamanho ímpar ou a média aritmética dos dois valores centrais em caso de arranjo com tamanho par.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código base fornecido pelo professor, com algumas modificações e adições. Vamos analisar se o aluno preencheu os campos obrigatórios e se adicionou uma descrição adequada.

**Nome, Matrícula e Data:**
- **Nome:** Maria Fernanda
- **Matrícula:** 894576
- **Data:** 28/10/2025

Esses campos foram preenchidos corretamente.

**Descrição do Programa:**
O aluno adicionou uma breve descrição no cabeçalho do código:
```python
# Substitua esta linha por um breve comentário dizendo o que o programa faz 
```
A descrição do programa não foi alterada, então não há uma descrição específica fornecida pelo aluno.

### Boas Práticas

Aqui estão as observações do pylint, reformuladas de forma resumida e em português:

1. **Nomes de Variáveis e Constantes:**
   - Os nomes das constantes `linf` e `lsup` devem seguir o estilo UPPER_CASE, pois são constantes. Por exemplo, `LINF` e `LSUP`.

2. **Espaços em Branco:**
   - Existem vários espaços em bra

###Prática 9

####Primeiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code9-1.py", "Solucao/solucao-code9.py", "lint/code9-1-lint.pre", "ast/code9-1-ast.pre", "Esqueleto/esqueleto-code9.py", "Saida/saida9.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code9.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code9.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p10.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code9.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code9-1.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code9-1-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code9-1-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida9.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve percorrer o arranjo bidimensional e atribuir um valor para cada elemento da matriz, sendo a parte inteira do valor = ao número da linha e a parte fracionária = ao número da coluna.
    - Criar um arranjo bidimensional B, de números inteiros e de mesma dimensão de A, seguindo a lógica descrita em um pseudocódigo no roteiro, imprimindo esse arranjo depois.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno segue o formato básico fornecido pelo professor, incluindo a importação da biblioteca `numpy` e a solicitação da dimensão da matriz. No entanto, há algumas observações importantes:

1. **Nome, Matrícula e Data**: O aluno preencheu os campos de nome, matrícula e data corretamente.
   
2. **Breve Descrição**: O aluno adicionou uma breve descrição do que o programa faz, indicando que "Esse programa cria duas matrizes A e B de dimensão N, completando de acordo com o pseudocódigo".

### Boas Práticas

1. **Trailing Whitespace**: O código contém espaços em branco no final de algumas linhas, o que pode ser removido para manter a clareza e a consistência do código.
   
2. **Docstrings**: Embora não seja um erro, é uma boa prática adicionar docstrings para documentar o que o módulo faz, suas funções e classes, e qualquer outro aspecto relevante.

3. **Nome do Módulo**: O nome do módulo (`code9-1.py`) não segue a convenção de nomes em snake_case. É re

####Segundo código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code9-2.py", "Solucao/solucao-code9.py", "lint/code9-2-lint.pre", "ast/code9-2-ast.pre", "Esqueleto/esqueleto-code9.py", "Saida/saida9.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code9.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code9.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p10.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code9.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code9-2.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code9-2-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code9-2-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida9.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve percorrer o arranjo bidimensional e atribuir um valor para cada elemento da matriz, sendo a parte inteira do valor = ao número da linha e a parte fracionária = ao número da coluna.
    - Criar um arranjo bidimensional B, de números inteiros e de mesma dimensão de A, seguindo a lógica descrita em um pseudocódigo no roteiro, imprimindo esse arranjo depois.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno Carla Maria foi baseado no código fornecido pelo professor, com algumas modificações e adições. Vamos analisar se o aluno preencheu os campos solicitados no código base:

1. **Nome do aluno**: O campo foi preenchido com "Carla Maria".
2. **Matrícula**: O campo foi preenchido com "235748".
3. **Data**: O campo foi preenchido com "30/10/25".
4. **Breve descrição do programa**: O aluno adicionou a descrição "São duas matrizes quadradas: a primeira com valores da linha e coluna e a segunda com valores definidos pela diagonal principal."

### Boas Práticas

Aqui estão as observações apontadas pelo pylint, adaptadas para uma linguagem mais amigável:

1. **Trailing whitespace**: Existem espaços em branco no final de algumas linhas. É recomendável remover esses espaços para manter um código limpo e evitar problemas futuros.
2. **Missing module docstring**: O código não possui uma descrição geral do que o módulo faz. Adicionar uma docstring no início

####Terceiro código

In [ ]:
from PyPDF2 import PdfReader

files = ["Codigo_aluno/code9-3.py", "Solucao/solucao-code9.py", "lint/code9-3-lint.pre", "ast/code9-3-ast.pre", "Esqueleto/esqueleto-code9.py", "Saida/saida9.txt"]
codes = {}

for file_path in files:
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        codes[file_path] = f.read()

pdf_path = "Roteiro/roteiro-code9.pdf"
pdf_text = ""

with open(pdf_path, "rb") as f:
    reader = PdfReader(f)
    for page in reader.pages:
        pdf_text += page.extract_text() or ""

new_prompt = f"""
Tenha como base a seguinte atividade de Introdução à programação:

{pdf_text}

E essa possível solução dada pelo professor para resolver a atividade:

{codes['Solucao/solucao-code9.py']}

Para resolver a atividade, o professor fornece o seguinte código base, que o roteiro chama de p10.py, para o aluno completar:

{codes['Esqueleto/esqueleto-code9.py']}

O código a seguir foi feito por um dos alunos dessa disciplina:

{codes['Codigo_aluno/code9-3.py']}.

Dada também uma árvore abstrata de sintaxe acerca do código do aluno:

{codes['ast/code9-3-ast.pre']}

E uma análise do pylint acerca do código do aluno:

{codes['lint/code9-3-lint.pre']}

Analise o código feito pelo aluno da disciplina e forneça um feedback explicativo e didático indicando erros e possíveis melhorias.

Para fazer isso, gere quatro seções: uma para descrição, uma para boas práticas, uma para a lógica empregada e outra para a formatação de saída.

    Para a seção de descrição:

        - Observe que, no código base dado pelo professor para o aluno completar, há comentários no topo para que o aluno possa preencher o nome, matrícula e data e adicionar uma breve descrição do que o programa faz.

        - Indique se o aluno preencheu o campo nome, matrícula e data; você pode fazer isso comparando se esses trechos estão diferentes no código que o aluno fez, partindo do código base.

        - Indique se o aluno adicionou uma breve descrição do que o programa faz; você pode fazer isso vendo se o comentário do código base está diferente do comentário do código do aluno.

    Para a seção de boas práticas:

        - Indique ao aluno todas as observações apontadas pelo pylint, mas não cite as observações com os códigos C0103 relativo ao nome do módulo, C0303 e C0114, caso estejam presentes.

        - Se o modelo for citar algo do pylint, reescreva resumidamente e em português, sem atribuir a ferramenta e sem usar as mensagens literais geradas pelo pylint.

        - Não é necessário fazer comentários acerca de observações do pylint de estruturas que já estavam presentes no código base fornecido pelo professor; avalie apenas o que o aluno implementou.

    Para a seção de lógica e concordância com o roteiro:

        - A implementação do professor deve servir apenas como base para sua análise e não como a única solução correta, podendo o aluno ter seguido outra lógica, também correta e funcional. Por isso, tome cuidado para não indicar erro em caso de uma operação que poderá gerar o mesmo resultado que a solução do professor mas que foi implementado de forma diferente.

        - Por se tratar de alunos iniciantes em programação, não trabalhamos com tratamento de erros; por isso, não faça comentários referentes a esse tópico.

        - Não retorne um código completo ajustado ou corrigido para o aluno mas sim fale bem detalhadamente sobre erros e melhorias.

        - Caso o aluno não tenha cumprido todas as tarefas definidas pelo roteiro ou tenha feito algo que não foi pedido pelo roteiro, faça comentários relativos a isso.

        - Não faça comentários sobre a formatação de saída nessa seção.

    Para a seção de formatação de saída:

        - Essa é a saída esperada pelo professor em um caso de teste de exemplo:

          {codes['Saida/saida9.txt']}

        - Compare a saída que será gerada pelo código do aluno com a saída esperada pelo professor: o código do aluno que foi considerado por você deve estar com as exatas palavras da saída esperada pelo professor e com os mesmos espaços em branco.

Para essa atividade, é importante lembrar que o roteiro define que:

    - O aluno deve percorrer o arranjo bidimensional e atribuir um valor para cada elemento da matriz, sendo a parte inteira do valor = ao número da linha e a parte fracionária = ao número da coluna.
    - Criar um arranjo bidimensional B, de números inteiros e de mesma dimensão de A, seguindo a lógica descrita em um pseudocódigo no roteiro, imprimindo esse arranjo depois.
"""

messages = [
    {"role": "system", "content": "Você é Qwen, um assistente que ajuda a analisar código."},
    {"role": "user", "content": new_prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048,
     do_sample=False
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Descrição

O código fornecido pelo aluno foi baseado no código inicial fornecido pelo professor. Vamos analisar se o aluno preencheu os campos solicitados:

1. **Nome do aluno:** O aluno preencheu o campo com "Sabrina Guedes".
2. **Matrícula:** O aluno preencheu o campo com "987456".
3. **Data:** O aluno preencheu o campo com "25/10/2025".
4. **Breve descrição do programa:** O aluno não substituiu o comentário do código base, mantendo-o como "Substitua esta linha por um breve comentário dizendo o que o programa faz".

### Boas Práticas

Aqui estão as observações do pylint, adaptadas para uma linguagem mais simples e direta:

1. **Trailing whitespace:** Existem espaços em branco no final de algumas linhas. É importante remover esses espaços para manter o código limpo.
2. **Trailing newlines:** O arquivo tem uma nova linha extra no final. É recomendável remover essa linha extra.
3. **Missing module docstring:** O código não possui uma descrição geral do que o programa faz. É importan